# 工具調用代理

## 概述

本教學說明 LangChain 中的工具調用，允許模型檢測何時**調用一個或多個工具以及傳遞什麼輸入**給這些工具。

在進行 API 調用時，您可以定義工具並智能地引導模型生成結構化物件，例如包含調用這些工具參數的 JSON。

工具 API 的目標是提供比標準文字完成或聊天 API 更可靠的有效且有用的**工具調用**生成。

您可以透過將此結構化輸出與將多個工具綁定到工具調用聊天模型的能力整合，並讓模型選擇要調用哪些工具，來創建反覆調用工具並接收結果直到查詢解決的代理。

這代表了 OpenAI 工具代理的更**通用版本**，該代理專門為 OpenAI 的特定工具調用風格而設計。

此代理使用 LangChain 的 ToolCall 介面來支援超越 OpenAI 之外更廣泛的供應商實現，包括 ```Anthropic```、```Google Gemini``` 和 ```Mistral```。

### 目錄

- [概述](#概述)
- [環境設定](#環境設定)
- [創建工具](#創建工具)
- [構建代理提示](#構建代理提示)
- [創建代理](#創建代理)
- [代理執行器](#代理執行器)
- [使用串流輸出檢查逐步結果](#使用串流輸出檢查逐步結果)
- [使用使用者定義函式自定義中間步驟輸出](#使用使用者定義函式自定義中間步驟輸出)
- [與先前對話歷史記錄的代理通訊](#與先前對話歷史記錄的代理通訊)

### 參考資料

- [LangChain Python API Reference > langchain: 0.3.14 > agents > create_tool_calling_agent](https://python.langchain.com/api_reference/langchain/agents/langchain.agents.tool_calling_agent.base.create_tool_calling_agent.html#create-tool-calling-agent)
- [LangChain Python API Reference > langchain: 0.3.14 > core > runnables > langchain_core.runnables.history > RunnableWithMessageHistory](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html)

![](./assets/15-agent-agent-concept.png)

---

## 我的見解

工具調用代理是現代 AI 應用的核心架構之一，它實現了 AI 從純文字生成到可以執行實際行動的重大跨越。這種設計讓 AI 能夠主動選擇和使用工具，而非被動地回應，大幅提升了 AI 系統的實用性和自主性。

LangChain 對多供應商的支援特別重要，這避免了供應商鎖定問題，讓開發者可以根據不同需求選擇最適合的模型。

## 學習補充重點

**核心概念理解：**
- **工具調用 vs 函式調用**：工具調用更廣泛，包含參數驗證、錯誤處理等
- **結構化輸出**：確保 AI 輸出符合預定義格式，提高可靠性
- **反覆調用機制**：AI 可以根據結果決定下一步行動

**多供應商優勢：**
- **成本優化**：不同模型的定價策略差異
- **功能特化**：某些模型在特定任務上表現更佳
- **風險分散**：避免單一供應商依賴

**實際應用場景：**
- **客服機器人**：查詢訂單、處理退款、更新資訊
- **資料分析助手**：執行 SQL 查詢、生成圖表、計算統計
- **內容管理**：搜尋文檔、更新記錄、發送通知
- **自動化工作流**：根據條件執行不同操作序列

**架構設計重點：**
- **工具選擇策略**：如何讓 AI 選擇最合適的工具
- **錯誤恢復機制**：工具調用失敗時的處理策略
- **狀態管理**：在多步驟操作中維護上下文

**效能考量：**
- **並行調用**：某些情況下可同時調用多個工具
- **快取機制**：避免重複的工具調用
- **超時處理**：防止工具調用無限等待

**安全性注意事項：**
- **工具權限控制**：限制 AI 可使用的工具範圍
- **輸入驗證**：防止惡意參數傳遞
- **審計日誌**：記錄所有工具調用行為

**除錯與監控：**
- **中間步驟追蹤**：了解 AI 的決策過程
- **工具使用統計**：分析工具使用模式
- **效能指標監控**：追蹤響應時間和成功率

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- ```langchain-opentutorial``` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

True

In [3]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_openai",
        "langchain_community",
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "ToolCallingAgent",
    }
)

Environment variables have been set successfully.


You can alternatively set ```OPENAI_API_KEY``` in ```.env``` file and load it.

[Note] This is not necessary if you've already set ```OPENAI_API_KEY``` in previous steps.

In [4]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

## Creating Tools

LangChain allows you to define custom tools that your agents can interact with. You can create tools for searching news or executing Python code.

The ```@tool``` decorator is used to create tools:
- ```TavilySearchResults``` is a tool for searching news.
- ```PythonREPL``` is a tool for executing Python code.


In [4]:
from langchain.tools import tool
from typing import List, Dict, Annotated
from langchain_community.tools import TavilySearchResults
from langchain_experimental.utilities import PythonREPL


# Creating tool for searching news
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search news by input keyword using Tavily Search API"""
    news_tool = TavilySearchResults(
        max_results=3,
        include_answer=True,
        include_raw_content=True,
        include_images=True,
        # search_depth="advanced",
        # include_domains = [],
        # exclude_domains = []
    )
    return news_tool.invoke(query, k=3)


# Creating tool for executing python code
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this tool to execute Python code. If you want to see the output of a value,
    you should print it using print(...). This output is visible to the user."""
    result = ""
    try:
        result = PythonREPL().run(code)
    except BaseException as e:
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        return result


print(f"Tool name: {search_news.name}")
print(f"Tool description: {search_news.description}")
print(f"Tool name: {python_repl_tool.name}")
print(f"Tool description: {python_repl_tool.description}")

Tool name: search_news
Tool description: Search news by input keyword using Tavily Search API
Tool name: python_repl_tool
Tool description: Use this tool to execute Python code. If you want to see the output of a value,
    you should print it using print(...). This output is visible to the user.


In [5]:
# Creating tools
tools = [search_news, python_repl_tool]

## 構建代理提示

- ```chat_history```：如果您的代理支援多輪對話，此變數會儲存對話歷史記錄。（否則，您可以省略此項。）
- ```agent_scratchpad```：此變數作為中間變數的暫存儲存空間。
- ```input```：此變數代表使用者的輸入。

---

## 我的見解

代理提示的構建是工具調用代理成功運作的關鍵基礎。這三個核心變數各自承擔不同的職責，共同形成代理的「思考框架」。特別是 `agent_scratchpad` 的設計很巧妙，它讓代理能夠進行「內部對話」，記錄推理過程和中間結果。

這種結構化的提示設計不僅提高了代理的推理能力，也使得除錯和優化變得更加容易。

## 學習補充重點

**變數功能詳解：**

**chat_history（對話歷史）：**
- 維持多輪對話的上下文連貫性
- 讓代理能參考先前的互動內容
- 支援複雜的對話流程和任務延續
- 可以包含使用者訊息、代理回應和工具調用結果

**agent_scratchpad（代理草稿區）：**
- 儲存工具調用的結果和中間推理
- 記錄代理的「思考過程」
- 幫助代理決定下一步行動
- 提供除錯和監控的重要資訊

**input（使用者輸入）：**
- 當前回合的使用者查詢
- 觸發代理行動的起始點
- 可以是問題、指令或資料

**提示模板設計模式：**
```python
# 基本模板結構
template = """
你是一個有用的助手，可以使用以下工具：
{tools}

請根據使用者輸入選擇合適的工具來回答問題。

對話歷史：
{chat_history}

目前任務：{input}

工作記錄：
{agent_scratchpad}
"""
```

**最佳實務建議：**
- **清晰的角色定義**：明確代理的身份和能力
- **工具使用指導**：說明何時和如何使用工具
- **格式化要求**：指定輸出格式和結構
- **錯誤處理指引**：告訴代理如何處理異常情況

**進階技巧：**
- **動態提示調整**：根據對話狀態調整提示內容
- **上下文壓縮**：當歷史記錄過長時進行智能摘要
- **任務分解指導**：教導代理如何拆解複雜任務
- **回饋循環**：利用 scratchpad 進行自我反思

**常見問題處理：**
- **記憶體管理**：避免 chat_history 過度膨脹
- **上下文混淆**：確保不同輪次的資訊不會相互干擾
- **工具選擇混亂**：提供明確的工具使用判斷標準

**除錯策略：**
- 檢查 agent_scratchpad 的內容了解推理過程
- 分析 chat_history 確認上下文是否正確傳遞
- 驗證 input 解析是否符合預期

In [6]:
from langchain_core.prompts import ChatPromptTemplate

# Creating prompt
# Prompt is a text that describes the task the model should perform. (input the name and role of the tool)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `search_news` tool for searching keyword related news.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

## Creating Agent

Define an agent using the ```create_tool_calling_agent``` function.

In [20]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent

# Creating LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Creating Agent
agent = create_tool_calling_agent(llm, tools, prompt)

## ```AgentExecutor```

```AgentExecutor``` 是用於管理使用工具的代理的類別。

**主要屬性**
- ```agent```：負責建立計劃並在執行迴圈的每個步驟中確定行動的底層代理。
- ```tools```：包含代理授權使用的所有有效工具的清單。
- ```return_intermediate_steps```：布林標誌，決定是否連同最終輸出一起返回代理採取的中間步驟。
- ```max_iterations```：代理在執行迴圈終止前可以採取的最大步驟數。
- ```max_execution_time```：允許執行迴圈運行的最大時間量。
- ```early_stopping_method```：定義當代理未返回 ```AgentFinish``` 時如何處理情況的方法。（「force」或「generate」）
  - ```"force"```：返回一個字串，指示執行迴圈因達到時間或迭代限制而停止。
  - ```"generate"```：呼叫代理的 LLM 鏈一次，根據先前採取的步驟生成最終答案。
- ```handle_parsing_errors```：指定如何處理解析錯誤。（您可以設定 ```True```、```False```，或提供自定義錯誤處理函式。）
- ```trim_intermediate_steps```：修剪中間步驟的方法。（您可以設定 ```-1``` 保留所有步驟，或提供自定義修剪函式。）

**主要方法**
1. ```invoke```：執行代理。
2. ```stream```：串流到達最終輸出所需的步驟。

---

## 我的見解

```AgentExecutor``` 是代理的執行引擎，提供完整的生命週期管理和資源控制。

## 學習補充重點

**關鍵配置：**
- ```max_iterations```：控制步驟上限，避免無限循環
- ```early_stopping_method```：選擇 "force"（強制停止）或 "generate"（生成答案）
- ```return_intermediate_steps```：除錯時設為 True

**實用技巧：**
- 簡單任務用較少迭代數（3-5次）
- 複雜任務可設定更多迭代（10-20次）
- 使用 ```stream``` 方法監控長時間任務

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [21]:
from langchain.agents import AgentExecutor

# Create AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=5,
    max_execution_time=5,
    handle_parsing_errors=True,
)

# Run AgentExecutor
result = agent_executor.invoke({"input": "Search news about AI Agent in 2025."})

print("Agent execution result:")
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `search_news` with `{'query': 'AI Agent 2025'}`


[{'url': 'https://www.biz4group.com/blog/ai-agent-use-cases', 'content': 'Across industries, AI agents are emerging as a transformative force in 2025, driving the next wave of intelligent automation. These autonomous software systems, powered by machine learning (ML), natural language processing (NLP), and large language models (LLMs), are capable of performing tasks, making decisions, and collaborating with users and tools with minimal manual input. [...] Market Expansion: The AI agents market is projected to grow from $7.38 billion in 2025 to $47.1 billion by 2030, reflecting a compound annual growth rate (CAGR) of 44.8%.\n   Enterprise Integration: Deloitte forecasts that by 2025, 25% of companies utilizing generative AI will initiate agentic AI pilots or proofs of concept, with this figure rising to 50% by 2027. [...] 2025 marks a major turning point for AI agents as real-world adop

## 使用串流輸出檢查逐步結果

我們將使用 ```AgentExecutor``` 的 ```stream()``` 方法來串流代理的中間步驟。

```stream()``` 的輸出在（行動、觀察）對之間交替，最後如果達成目標則以代理的答案結束。

流程如下所示：

1. 行動輸出
2. 觀察輸出
3. 行動輸出
4. 觀察輸出

...（持續直到達成目標）...

然後，如果代理達成目標，將得出最終答案。

下表總結了您在輸出中會遇到的內容：

| 輸出 | 描述 |
|--------|-------------|
| 行動 | ```actions```：代表 ```AgentAction``` 或其子類別。<br>```messages```：與行動調用對應的聊天訊息。 |
| 觀察 | ```steps```：代理工作的記錄，包括當前行動及其觀察。<br>```messages```：包含函式調用結果（即觀察）的聊天訊息。 |
| 最終答案 | ```output```：代表 ```AgentFinish``` 信號。<br>```messages```：包含最終輸出的聊天訊息。 |

---

## 我的見解

串流輸出提供了代理「思考過程」的即時可見性，這對除錯和用戶體驗都很重要。行動-觀察的循環模式清楚展現了代理的推理邏輯。

## 學習補充重點

**串流的實際價值：**
- **即時反饋**：用戶可以看到進度，不會感到系統卡住
- **除錯利器**：開發者能追蹤每個決策點
- **早期介入**：發現錯誤方向時可及時停止

**常見輸出模式：**
```
Action → 工具調用（如搜尋、計算）
Observation → 工具返回的結果
Action → 基於結果的下一步行動
Observation → 新的結果
...
Final Answer → 最終回答用戶
```

**最佳實務：**
- 長任務必須使用串流避免超時
- 生產環境可隱藏中間步驟，只顯示最終結果
- 記錄完整串流過程便於後續分析

In [22]:
from langchain.agents import AgentExecutor

# Create AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
    handle_parsing_errors=True,
)

In [23]:
# Run in streaming mode
result = agent_executor.stream({"input": "Search news about AI Agent in 2025."})

for step in result:
    # Print intermediate steps
    print(step)
    print("===" * 20)

{'actions': [ToolAgentAction(tool='search_news', tool_input={'query': 'AI Agent 2025'}, log="\nInvoking: `search_news` with `{'query': 'AI Agent 2025'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_v99Fp6cNjUNCUO6OhwEWekfw', 'function': {'arguments': '{"query":"AI Agent 2025"}', 'name': 'search_news'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'service_tier': 'default'}, id='run--9fcb3a11-4ba5-44e5-b8b1-7a5ec218dcaf', tool_calls=[{'name': 'search_news', 'args': {'query': 'AI Agent 2025'}, 'id': 'call_v99Fp6cNjUNCUO6OhwEWekfw', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'search_news', 'args': '{"query":"AI Agent 2025"}', 'id': 'call_v99Fp6cNjUNCUO6OhwEWekfw', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_v99Fp6cNjUNCUO6OhwEWekfw')], 'messages': [AIMessageChunk(content='', additional_kwargs={'too

## 使用使用者定義函式自定義中間步驟輸出

您可以定義以下 3 個函式來自定義中間步驟輸出：

- ```tool_callback```：此函式處理工具調用產生的輸出。
- ```observation_callback```：此函式處理觀察資料輸出。
- ```result_callback```：此函式允許您處理最終答案輸出。

---

## 我的見解

這種回調函式設計讓開發者能完全控制代理輸出的展示方式，非常適合客製化用戶介面或整合到現有系統中。

## 學習補充重點

**回調函式的應用場景：**
- **UI 整合**：將輸出格式化為特定 UI 元件
- **日誌記錄**：記錄每個步驟的詳細資訊
- **進度追蹤**：更新進度條或狀態指示器
- **錯誤處理**：在特定步驟失敗時觸發警報

**典型實現範例：**
```python
def tool_callback(action):
    print(f"🔧 使用工具: {action.tool}")
    
def observation_callback(observation):
    print(f"👁️ 觀察結果: {observation[:100]}...")
    
def result_callback(result):
    print(f"✅ 最終答案: {result}")
```

**進階技巧：**
- 可以在回調中進行資料預處理
- 結合日誌系統記錄代理行為
- 實現條件性輸出（如只在錯誤時顯示詳情）

Here's an example callback function that demonstrates how to clean up the intermediate steps of the Agent.

This callback function can be useful when presenting intermediate steps to users in an application like Streamlit.

In [24]:
from typing import Dict, Any


# Create AgentStreamParser class
class AgentStreamParser:
    def __init__(self):
        pass

    def tool_callback(self, tool: Dict[str, Any]) -> None:
        print("\n=== Tool Called ===")
        print(f"Tool: {tool.get('tool')}")
        print(f"Input: {tool.get('tool_input')}")
        print("==================\n")

    def observation_callback(self, step: Dict[str, Any]) -> None:
        print("\n=== Observation ===")
        observation_data = step["steps"][0].observation
        print(f"Observation: {observation_data}")
        print("===================\n")

    def result_callback(self, result: str) -> None:
        print("\n=== Final Answer ===")
        print(result)
        print("====================\n")

    def process_agent_steps(self, step: Dict[str, Any]) -> None:
        if "actions" in step:
            for action in step["actions"]:
                self.tool_callback(
                    {"tool": action.tool, "tool_input": action.tool_input}
                )
        elif "output" in step:
            self.result_callback(step["output"])
        else:
            self.observation_callback(step)


# Create AgentStreamParser instance
agent_stream_parser = AgentStreamParser()

Check the response process of your Agent in streaming mode.

In [25]:
# Run in streaming mode
result = agent_executor.stream({"input": "Generate a pie chart using matplotlib."})
# result = agent_executor.stream({"input": "Search news about AI Agent in 2025."})


for step in result:
    agent_stream_parser.process_agent_steps(step)

Python REPL can execute arbitrary code. Use with caution.



=== Tool Called ===
Tool: python_repl_tool
Input: {'code': "import matplotlib.pyplot as plt\n\n# Data to plot\nlabels = ['A', 'B', 'C', 'D']\nsizes = [15, 30, 45, 10]\ncolors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue']\nexplode = (0.1, 0, 0, 0)  # explode 1st slice\n\n# Plot\nplt.figure(figsize=(8, 6))\nplt.pie(sizes, explode=explode, labels=labels, colors=colors,\n        autopct='%1.1f%%', shadow=True, startangle=140)\nplt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.\nplt.title('Pie Chart Example')\nplt.show()"}


=== Observation ===
Observation: ModuleNotFoundError("No module named 'matplotlib'")


=== Final Answer ===
It seems that the environment does not have the `matplotlib` library available to generate the pie chart. However, I can provide you with the code that you can run in your own Python environment to create the pie chart.

Here is the code:

```python
import matplotlib.pyplot as plt

# Data to plot
labels = ['A', 'B', 'C', 'D']

In [14]:
# Run in streaming mode
result = agent_executor.stream({"input": "Search news about AI Agent in 2025."})


for step in result:
    agent_stream_parser.process_agent_steps(step)


=== Tool Called ===
Tool: search_news
Input: {'query': 'AI Agent 2025'}


=== Observation ===
Observation: [{'url': 'https://www.analyticsvidhya.com/blog/2024/12/ai-agent-trends/', 'content': 'In a similar study, Deloitte forecasts that 25% of enterprises using GenAI will deploy AI Agents by 2025, growing to 50% by 2027. Meanwhile, Gartner predicts that by 2028, at least 15% of day-to-day work decisions will be made autonomously through agentic AI. It also states that by then, 33% of enterprise software applications will also include'}, {'url': 'https://www.techtarget.com/searchEnterpriseAI/feature/Next-year-will-be-the-year-of-AI-agents', 'content': 'Next year will be the year of AI agents | TechTarget This will make the AI agent more accurate in completing its task, Greene said. Other than the rise of single-task AI agents, 2025 may also be the year of building the infrastructure for AI agents, said Olivier Blanchard, an analyst with Futurum Group. "2025 isn\'t going to be the year 

Modify the callback function to use it.

In [15]:
from typing import Dict, Any, Callable


# 1. Define AgentCallbacks class
class AgentCallbacks:
    def __init__(
        self,
        tool_callback: Callable,
        observation_callback: Callable,
        result_callback: Callable,
    ):
        self.tool_callback = tool_callback
        self.observation_callback = observation_callback
        self.result_callback = result_callback


# 2. Define AgentStreamParser class
class AgentStreamParser:
    def __init__(self, callbacks: AgentCallbacks):
        self.callbacks = callbacks

    def process_agent_steps(self, step: Dict[str, Any]) -> None:
        if "actions" in step:
            for action in step["actions"]:
                self.callbacks.tool_callback(
                    {"tool": action.tool, "tool_input": action.tool_input}
                )
        elif "output" in step:
            self.callbacks.result_callback(step["output"])
        else:
            self.callbacks.observation_callback(step)


# 3. Define callback functions
def tool_callback(tool) -> None:
    print("<<<<<<< Tool Called >>>>>>")
    print(f"Tool: {tool.get('tool')}")
    print(f"Input: {tool.get('tool_input')}")
    print("<<<<<<< Tool Called >>>>>>")


def observation_callback(step) -> None:
    print("<<<<<<< Observation >>>>>>")
    observation_data = step["steps"][0].observation
    print(f"Observation: {observation_data}")
    print("<<<<<<< Observation >>>>>>")


def result_callback(result: str) -> None:
    print("<<<<<<< Final Answer >>>>>>")
    print(result)
    print("<<<<<<< Final Answer >>>>>>")


# 4. Example usage
# Wrap callback functions into AgentCallbacks instance
agent_callbacks = AgentCallbacks(
    tool_callback=tool_callback,
    observation_callback=observation_callback,
    result_callback=result_callback,
)

# Create AgentStreamParser instance
agent_stream_parser = AgentStreamParser(agent_callbacks)

Check the output content. You can reflect the output value of your callback functions, providing intermediate content that has been changed.

In [16]:
# Request streaming output for the query
result = agent_executor.stream({"input": "Search news about AI Agent in 2025."})

for step in result:
    # Output intermediate steps using parser
    agent_stream_parser.process_agent_steps(step)

<<<<<<< Tool Called >>>>>>
Tool: search_news
Input: {'query': 'AI Agent 2025'}
<<<<<<< Tool Called >>>>>>
<<<<<<< Observation >>>>>>
Observation: [{'url': 'https://www.analyticsvidhya.com/blog/2024/12/ai-agents-to-look-out-for/', 'content': "Q2. Why are these five AI agents considered game-changers for 2025? Ans. The selected agents—Oracle's Miracle Agent, Nvidia's Eureka Agent, Google's Project Jarvis, SAP's Joule Collaborative AI Agents, and Cisco's Webex AI Agent—stand out due to their innovative designs, broad applications, and industry impact."}, {'url': 'https://www.analyticsvidhya.com/blog/2024/12/ai-agent-trends/', 'content': 'In a similar study, Deloitte forecasts that 25% of enterprises using GenAI will deploy AI Agents by 2025, growing to 50% by 2027. Meanwhile, Gartner predicts that by 2028, at least 15% of day-to-day work decisions will be made autonomously through agentic AI. It also states that by then, 33% of enterprise software applications will also include'}, {'url':

## Communicating Agent with previous conversation history

To remember past conversations, you can wrap the ```AgentExecutor``` with ```RunnableWithMessageHistory```.

For more details on ```RunnableWithMessageHistory```, please refer to the link below.

**Reference**
- [LangChain Python API Reference > langchain: 0.3.14 > core > runnables > langchain_core.runnables.history > RunnableWithMessageHistory](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html)

In [26]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Create a dictionary to store session_id
store = {}


# Function to get session history based on session_id
def get_session_history(session_ids):
    if session_ids not in store:  # If session_id is not in store
        # Create a new ChatMessageHistory object and store it in store
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # Return session history for the corresponding session_id


# Create an agent with chat message history
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # Chat session_id
    get_session_history,
    # The key for the question input in the prompt: "input"
    input_messages_key="input",
    # The key for the message input in the prompt: "chat_history"
    history_messages_key="chat_history",
)

In [27]:
# Request streaming output for the query
response = agent_with_chat_history.stream(
    {"input": "Hello! My name is Teddy!"},
    # Set session_id
    config={"configurable": {"session_id": "abc123"}},
)

# Check the output
for step in response:
    agent_stream_parser.process_agent_steps(step)


=== Final Answer ===
Hello, Teddy! How can I assist you today?



In [28]:
# Request streaming output for the query
response = agent_with_chat_history.stream(
    {"input": "What is my name?"},
    # Set session_id
    config={"configurable": {"session_id": "abc123"}},
)

# Check the output
for step in response:
    agent_stream_parser.process_agent_steps(step)


=== Final Answer ===
Your name is Teddy! How can I help you today?



In [29]:
# Request streaming output for the query
response = agent_with_chat_history.stream(
    {
        "input": "My email address is teddy@teddynote.com. The company name is TeddyNote Co., Ltd."
    },
    # Set session_id
    config={"configurable": {"session_id": "abc123"}},
)

# Check the output
for step in response:
    agent_stream_parser.process_agent_steps(step)


=== Final Answer ===
Thank you for sharing that information, Teddy! How can I assist you with TeddyNote Co., Ltd. or anything else today?



In [30]:
# Request streaming output for the query
response = agent_with_chat_history.stream(
    {
        "input": "Search the latest news and write it as the body of the email. "
        "The recipient is `Ms. Sally` and the sender is my personal information."
        "Write in a polite tone, and include appropriate greetings and closings at the beginning and end of the email."
    },
    # Set session_id
    config={"configurable": {"session_id": "abc123"}},
)

# Check the output
for step in response:
    agent_stream_parser.process_agent_steps(step)


=== Tool Called ===
Tool: search_news
Input: {'query': 'latest news'}


=== Observation ===
Observation: [{'url': 'https://en.wikipedia.org/wiki/Portal:Current_events/June_2025', 'content': 'Arts and culture\n\n   Argentine presidentJavier Milei is awarded the Genesis Prize during a state visit to Israel for his support for Israel, becoming the first non-Jewish person to receive such recognition. (_Clarín_)(_Haaretz_)\n\nBusiness and economy\n\n   Australianflag carrierQantas announces the closure of its Singapore-based, partly owned low-cost airlineJetstar Asia due to rising costs and regional competition. (BBC News)\n\nDisasters and accidents [...] 2025 Gerik bus crash\n       Fifteen people are killed and 33 others are injured in a traffic collision when a bus carrying students loses control and crashes into the back of a minivan on the East–West Highway "East–West Highway (Malaysia)") near Gerik, Perak, Malaysia. (France 24) [...] Russian invasion of Ukraine\n       Kharkiv strike

In [31]:
# Request streaming output for the query
response = agent_with_chat_history.stream(
    {"input": "What is my name?"},
    # Set session_id
    config={"configurable": {"session_id": "def456"}},
)

# Check the output
for step in response:
    agent_stream_parser.process_agent_steps(step)


=== Final Answer ===
I don't have access to your personal information, including your name. If you'd like to share your name or any other details, feel free to do so!

